# Homework 3 Code

In [ ]:
# Add import statements here
import numpy as np
import statsmodels.api as sm
from scipy import stats
from matplotlib import pyplot as plt
import math
import time
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# To access files in your Google Drive, run this block and follow the instructions
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# To test if the above block worked, run this block
!ls '/content/gdrive/My Drive/'

 ## Find test error

The `find_test_error` function computes the test error of a linear classifier $w$. 

The hypothesis is assumed to be of the form $sign([1, x(N,:)] \cdot w)$.

Inputs:
* `w` is the weight vector
* `X` is the data matrix (without an initial column of 1's)
* `y` are the data labels (plus or minus 1)

Outputs:
* `test_error` is the binary error of $w$ on the data set $(X, y)$ error; this should be between 0 and 1. 

In [ ]:
def find_test_error(w, X, y):

  # Your code here, assign the proper value to test_error:
  temp = np.ones((np.size(X,0), 1), dtype = float)
  X = np.concatenate((temp, X), axis = 1)
  Xw = np.dot(X,w)
  tlabel = np.sign(Xw)
  # print('tlabel', tlabel)
  sumarray = np.ones((np.size(X,0), 1), dtype = float)
  for i in range(np.size(X,0)):
    if y[i] == tlabel[i]:
      sumarray[i] = 0
    else:
      sumarray[i] = sumarray[i] / np.size(X,0)
  test_error = sum(sumarray)
  return test_error

 ## Logistic Regression

The `logistic_reg`  learn a logistic regression model using gradient descent.

Inputs:
* `X` is the data matrix (without an initial column of 1's)
* `y` are the data labels (plus or minus 1)
* `w_init` is the initial value of the w vector ($d+1$ dimensional)
* `max_its` is the maximum number of iterations to run for
* `eta` is the learning rate

Outputs:
* t is the number of iterations gradient descent ran for
* w is the learned weight vector
* e_in is the in-sample (cross-entropy) error 

In [ ]:
def logistic_reg(X, y, w_init, max_its, eta):

  # Your code here, assign the proper values to t, w, and e_in:
  startT = time.time()
  onesArr = np.ones((np.size(X,0), 1), dtype = float)
  X = np.concatenate((onesArr, X), axis = 1)
  wold = w_init
  nitr = 0
  n = np.size(X,0)
  tol = 0.1
  wnew = np.array([])
  while tol > 0.001 and nitr < max_its:
    Xwold = np.dot(X, wold)
    yXwold = np.multiply(y, np.transpose(Xwold))
    denom = 1 + np.exp(yXwold)
    gval = np.multiply(np.transpose(X), y) / denom
    g = -gval.sum(axis=1) / n
    wnew = np.subtract(np.transpose(wold), eta*g)
    wnew = np.transpose(wnew)
    wold = wnew
    nitr = nitr + 1
    tol = np.absolute(g).max()
  print(nitr)
  w = wnew
  einval = np.log(1 + np.exp(-1 * np.multiply(y,np.transpose(np.dot(X, w)))))
  e_in = np.sum(einval) / n
  endT = time.time()
  t = endT - startT
  return t, w, e_in

Logistic Regression for part b

In [ ]:
def logistic_reg2(X, y, w_init, eta):
  # Your code here, assign the proper values to t, w, and e_in:
  startT = time.time()
  onesArr = np.ones((np.size(X,0), 1), dtype = float)
  X = np.concatenate((onesArr, X), axis = 1)
  wold = w_init
  nitr = 0
  n = np.size(X,0)
  tol = 0.1
  wnew = np.array([])
  while tol > 0.000001:
    Xwold = np.dot(X, wold)
    yXwold = np.multiply(y, np.transpose(Xwold))
    denom = 1 + np.exp(yXwold)
    gval = np.multiply(np.transpose(X), y) / denom
    g = -gval.sum(axis=1) / n
    wnew = np.subtract(np.transpose(wold), eta*g)
    wnew = np.transpose(wnew)
    wold = wnew
    nitr = nitr + 1
    tol = np.absolute(g).max()
  w = wnew
  einval = np.log(1 + np.exp(-1 * np.multiply(y,np.transpose(np.dot(X, w)))))
  e_in = np.sum(einval) / n
  endT = time.time()
  t = endT - startT
  return t, w, e_in, nitr

## Run and Plot

Run your code and plot figures below

In [ ]:
%xmode Plain
%pdb off
# Other code here:
#Question 1 part a
#importing csv files and converting them to numpy arrays
rawtrain = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CSVfiles/clevelandtrain.csv')
rawtest = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CSVfiles/clevelandtest.csv')
rawtrain = rawtrain.to_numpy()
rawtest = rawtest.to_numpy()

#creating x matrix and y arrray for test and train sets
xtrain = rawtrain[:, range(0,13)]
ytrain = rawtrain[:, 13]
xtest = rawtest[:, range(0,13)]
ytest = rawtest[:, 13]

#changing y from 1&0 to -1&1
#y = [2 * ph - 1 for ph in y]
ytrain = ymodifier(ytrain)
ytest = ymodifier(ytest)

#finding logistic regression
time10k, weight10k, ein10k = logistic_reg(xtrain, ytrain, np.zeros((14,1)), 10000, 0.00001)
print(time10k, ein10k)
time100k, weight100k, ein100k = logistic_reg(xtrain, ytrain, np.zeros((14,1)), 100000, 0.00001)
print(time100k, ein100k)
time1mil, weight1mil, ein1mil = logistic_reg(xtrain, ytrain, np.zeros((14,1)), 1000000, 0.00001)
print(time1mil, ein1mil)

#finding test error of train set
trainError10k = find_test_error(weight10k, xtrain, ytrain)
trainError100k = find_test_error(weight100k, xtrain, ytrain)
trainError1mil = find_test_error(weight1mil, xtrain, ytrain)
print(trainError10k)
print(trainError100k)
print(trainError1mil)

#finging test error of test set
testError10k = find_test_error(weight10k, xtest, ytest)
testError100k = find_test_error(weight100k, xtest, ytest)
testError1mil = find_test_error(weight1mil, xtest, ytest)
print(testError10k)
print(testError100k)
print(testError1mil)

Exception reporting mode: Plain
Automatic pdb calling has been turned OFF
10000
0.4043424129486084 0.5847145522443018
100000
4.098693132400513 0.49370175927710563
1000000
40.32430100440979 0.43535260915252544
[0.30921053]
[0.22368421]
[0.15131579]
[0.31724138]
[0.20689655]
[0.13103448]


In [ ]:
%xmode Plain
%pdb off
# Question 1 part b
rawtrain = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CSVfiles/clevelandtrain.csv')
rawtest = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CSVfiles/clevelandtest.csv')
rawtrain = rawtrain.to_numpy()
rawtest = rawtest.to_numpy()

# creating x matrix and y arrray for test and train sets
xtrain = rawtrain[:, range(0,13)]
ytrain = rawtrain[:, 13]
xtest = rawtest[:, range(0,13)]
ytest = rawtest[:, 13]

#changing y from 1&0 to -1&1
#y = [2 * ph - 1 for ph in y]
ytrain = ymodifier(ytrain)
ytest = ymodifier(ytest)

# calculating mean and sd of x matrix from training set
meanArr = np.mean(xtrain, axis = 0) 
sdArr = np.std(xtrain, axis = 0)

#calculating zscore different way with stats
# zXtrain = stats.zscore(xtrain, axis=0)

# scaling xtrain and xtest
scaleXtrain = (xtrain - meanArr) / sdArr
scaleXtest = (xtest - meanArr) / sdArr

# running logistic regression at different learning rates
lr = [0.01, 0.1, 1, 4, 5, 6, 7, 7.5, 7.6, 7.65]
timelr1, wlr1, einlr1, iterlr1 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[0])
print('learning rate 0.01', 'time', timelr1, 'e_in', einlr1, 'iterations', iterlr1)
timelr2, wlr2, einlr2, iterlr2 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[1])
print('learning rate 0.1', 'time', timelr2, 'e_in', einlr2, 'iterations', iterlr2)
timelr3, wlr3, einlr3, iterlr3 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[2])
print('learning rate 1', 'time', timelr3, 'e_in', einlr3, 'iterations', iterlr3)
timelr4, wlr4, einlr4, iterlr4 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[3])
print('learning rate 4', 'time', timelr4, 'e_in', einlr4, 'iterations', iterlr4)
timelr5, wlr5, einlr5, iterlr5 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[4])
print('learning rate 5', 'time', timelr5, 'e_in', einlr5, 'iterations', iterlr5)
timelr6, wlr6, einlr6, iterlr6 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[5])
print('learning rate 6', 'time', timelr6, 'e_in', einlr6, 'iterations', iterlr6)
timelr7, wlr7, einlr7, iterlr7 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[6])
print('learning rate 7', 'time', timelr7, 'e_in', einlr7, 'iterations', iterlr7)
timelr8, wlr8, einlr8, iterlr8 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[7])
print('learning rate 7.5', 'time', timelr8, 'e_in', einlr8, 'iterations', iterlr8)
timelr9, wlr9, einlr9, iterlr9 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[8])
print('learning rate 7.6', 'time', timelr9, 'e_in', einlr9, 'iterations', iterlr9)
timelr10, wlr10, einlr10, iterlr10 = logistic_reg2(scaleXtest, ytest, np.zeros((14,1)), lr[9])
print('learning rate 7.65', 'time', timelr10, 'e_in', einlr10, 'iterations', iterlr10)

# test and train errors
testErrorLr1 = find_test_error(wlr1, scaleXtest, ytest)
trainErrorLr1 = find_test_error(wlr1, scaleXtrain, ytrain)
print('trainError for lr = 0.01', trainErrorLr1, 'testError for lr = 0.01', testErrorLr1)
testErrorLr10 = find_test_error(wlr10, scaleXtest, ytest)
trainErrorLr10 = find_test_error(wlr10, scaleXtrain, ytrain)
print('trainError for lr = 7.65', trainErrorLr10, 'testError for lr = 7.65', testErrorLr10)

Exception reporting mode: Plain
Automatic pdb calling has been turned OFF
learning rate 0.01 time 2.976383686065674 e_in 0.23065415514539747 iterations 74909
learning rate 0.1 time 0.31060266494750977 e_in 0.23065415514489573 iterations 7487
learning rate 1 time 0.03164958953857422 e_in 0.23065415513855675 iterations 745
learning rate 4 time 0.008558034896850586 e_in 0.2306541551199055 iterations 180
learning rate 5 time 0.007134914398193359 e_in 0.23065415510966955 iterations 140
learning rate 6 time 0.00652623176574707 e_in 0.23065415511056597 iterations 109
learning rate 7 time 0.003779172897338867 e_in 0.23065415500165967 iterations 65
learning rate 7.5 time 0.005596160888671875 e_in 0.2306541550964868 iterations 83
learning rate 7.6 time 0.0052394866943359375 e_in 0.23065415509649165 iterations 84
learning rate 7.65 time 0.005105495452880859 e_in 0.23065415508144665 iterations 85
trainError for lr = 0.01 [0.19078947] testError for lr = 0.01 [0.10344828]
trainError for lr = 7.65 [0

In [ ]:
def ymodifier(y):
  ymod = 2*y-1
  return ymod